In [ ]:
!nvidia-smi

Mon Jun 21 20:07:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# for colab!!!
!git clone https://github.com/matrs01/SincNet-Implementation.git

Cloning into 'SincNet-Implementation'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
from typing import Optional, Tuple, Callable

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sys
from torch.autograd import Variable
import math
import soundfile as sf
from tqdm import tqdm

In [4]:
%cd /content/SincNet-Implementation

/content/SincNet-Implementation


In [5]:
from model import sinc_Conv1d, LayerNorm, SincNet
from utilities import SincNet_Dataset, Valid_DataLoader, train, ReadList

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [7]:
%%time
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1bJnP1rAuuJXjIqouvdH8awSgr-mH_kMT' , 
                                    dest_path='/content/TIMIT_preprocessed.zip', 
                                    unzip=True)

Unzipping...Done.
CPU times: user 4.28 s, sys: 1.46 s, total: 5.74 s
Wall time: 12 s


In [8]:
lr = 0.001
batch_size = 2400
N_eval_epoch = 8
seed = 1234

In [9]:
train_track_list = ReadList('data_lists/TIMIT_train.scp')
val_track_list = ReadList('data_lists/TIMIT_test.scp')
data_path = '/content/TIMIT_preprocessed/'
num_train_tracks = len(train_track_list)
num_val_tracks = len(val_track_list)

In [10]:
torch.manual_seed(seed)
np.random.seed(seed)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
input_dim = 3200
shift_size = 160

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [14]:
# Loading label dictionary
labels_dict = np.load('data_lists/TIMIT_labels.npy', allow_pickle=True).item()

In [15]:
model = SincNet(input_dim=input_dim).to(DEVICE)

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
train_dataset = SincNet_Dataset(batch_size, data_path, 
                                train_track_list, num_train_tracks, 
                                input_dim, labels_dict)

In [18]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = Valid_DataLoader(batch_size, data_path, 
                                val_track_list, num_val_tracks, 
                                input_dim, shift_size, labels_dict)

In [ ]:
train_losses, train_cerr, val_losses, val_cerr, val_sentece_cerr = train(model, 
                                                                         train_dataloader, 
                                                                         valid_dataloader, 
                                                                         optimizer, 
                                                                         criterion, 
                                                                         num_epochs=10000, 
                                                                         N_eval_epochs=400,
                                                                         save_path='/content/gdrive/MyDrive/Deep_Learning/Audio/SincNet/checkpoints/run007__bs=1200.pth')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


(epoch 0) VALID: loss = 6.135992551778818, cerr = 0.99769623301215, snt_cerr = 0.9956709956709957
(epoch 400) TRAIN: loss = 3.8831424629688263, cerr = 0.797831168831169
(epoch 400) VALID: loss = 3.6929122477898866, cerr = 0.7764044025187322, snt_cerr = 0.2049062049062049
(epoch 800) TRAIN: loss = 1.34682788297534, cerr = 0.3361428571428569
(epoch 800) VALID: loss = 3.412116493282552, cerr = 0.6730591885694133, snt_cerr = 0.05916305916305916
(epoch 1200) TRAIN: loss = 0.45513147555291655, cerr = 0.11695454545454548
(epoch 1200) VALID: loss = 3.7165515484532925, cerr = 0.6560198733775542, snt_cerr = 0.05411255411255411
(epoch 1600) TRAIN: loss = 0.18143391862511635, cerr = 0.04671536796536796
(epoch 1600) VALID: loss = 3.9844821681675713, cerr = 0.6412055058239615, snt_cerr = 0.05483405483405483
(epoch 2000) TRAIN: loss = 0.09432727382518351, cerr = 0.02415476190476191
(epoch 2000) VALID: loss = 4.194822079508765, cerr = 0.6385211615423559, snt_cerr = 0.05411255411255411
(epoch 2400) TRA

In [ ]:
# torch.save(model.state_dict(), '/content/gdrive/MyDrive/Deep_Learning/Audio/SincNet/checkpoints/bs=1000__num_epoch=1000__train_loss=0.51__val_los=3.34__val_cerr=0.64__val_cent_cer=0.077.pth')